 Using your programming language of choice, write a
function that computes the expected relative astrometry, absolute astrometry, and radial velocities
of a two-body system. That is, it should take the time, the orbital elements, and some way of
describing the object masses (either both masses or the mass ratio) as input. It should return: 1)
the on-sky projected separation and position angle of B with respect to A, 2) the on-sky projected
separation and position angle of both objects with respect to the center-of-mass position, and 3)
the RVs of both objects with respect to the center-of-mass RV.
Note that by returning all of these numbers, your new routine can flexibly serve as the core
of an orbit fitter for a wide range of data types: RVs, direct imaging (such as from AO imaging),
photocenter motion (such as from Gaia or Hipparcos), and transits. If you can think of something
else that sounds useful to return, feel free to implement that too.
You can adopt the appropriate equations from the Exoplanets textbook, but note that you need
to implement a solver for the transcendental equation. You don’t need to consider light travel times,
but at least keep in mind that we’re getting into a realm of data precision where that could matter.
(I actually had to do this for my recent paper on the young eclipsing binary UScoCTIO 5. It turns
out that Kepler data really is that good.

In [7]:
from math import *
import random as rand
import matplotlib.pyplot as plt
#from __future__ import division #uncomment if running in python 2.x

In [54]:
#orbital elements a,e,i,W,w,t0
G = 6.67e-11
auToMeters = 149597870700

def newtonsMethod(f, fprime, guess, tolerance=1e-4):
    curr, prev = guess, 2**31 -1
    while abs(curr - prev) > tolerance:
        prev, curr = curr,  curr - f(curr)/fprime(curr)
    return curr


def orbits(elementList, t, M1=0, M2=0, m1Overm2=0, VzBC=0):
    if len(elementList) != 6:
        raise(Exception("Expected 6 orbital elements in element list\n"))
    a, e, i, W, w, t0 = elementList
    i, W, w = i*pi/180, W*pi/180, w*pi/180 #convert to radians
    if not(M1 or M2 or m1Overm2):
        raise(Exception("At least one of the following must be nonzero M1, M2, m1Overm2\n"))
    if M1 or M2:
        T = 2*pi*a*sqrt(a/(G*(M1+M2)))
        M2frac = M2/(M1+M2)
    else:
        T = 2*pi*a*sqrt(a/(G*(1+m1Overm2)))
        M2frac = 1/(m1Overm2 + 1)
    n = 2*pi/T
    #print(T/86400)
    M = n*(t-t0)*86400
    g = lambda E: E - e*sin(E) - M
    gPrime = lambda E: 1 - e*cos(E)
    E0 = newtonsMethod(g, gPrime, M)
    r = a*(1-e*cos(E0))
    f = 2*atan2(sqrt(1+e)*sin(E0/2),sqrt(1-e)*cos(E0/2))
    theta = (f + w + W)*180/pi%360
    Xr = cos(W)*cos(w+f) - sin(W)*sin(w+f)*cos(i)
    Yr = sin(W)*cos(w+f) + cos(W)*sin(w+f)*cos(i)
    Zr = sin(w+f)*sin(i)
    Zr2 = Zr*Zr
    a1 = M2frac*a
    a2 = a - a1
    r1, r2 = a1*(1 - e*cos(E0)), a2*(1-e*cos(E0))
    k = n*a*sin(i)/sqrt(1-e*e)
    K1 = M2frac*k
    K2 = k - K1
    vr1 = K1*(cos(w+f)+e*cos(w)) + VzBC
    vr2 = K2*(cos(w+f)+e*cos(w+pi)) + VzBC
    r1/= auToMeters
    r2/= auToMeters
    f*=180/pi
    return [r/auToMeters*sqrt(1-Zr2), f], [r1*sqrt(1-Zr2),f, r1*Xr, r1*Yr, r1*Zr],\
[r2*sqrt(1-Zr2),(f+180)%360, r2*Xr, r2*Yr, r2*Zr], [-vr1/1000,vr2/1000]


    

The transiting planet HD 80606 b is notable for
being a transiting planet on a very extreme orbit, with eccentricity of e = 0.934. Go out into the
literature and find the orbital elements for this planet, and use your orbit prediction code to predict
the RV curve over the course of this semester. You should turn in a plot of stellar RV versus time
spanning August-December. If you were going to ask for telescope time on the McDonald 2.7m in
order to measure the extreme maximum and minimum, which nights would you want?


In [98]:
#auToMeters = 8.5*60*3e8
MSun = 1.988e30
MJupiter = 1.898e27

# ****************** ALL PLOTS ARE FOR HD 80606 b ********************
system = [0.449*auToMeters,0.93369, 89.32, 90,300.53,2454424.852]
l1,l2,l3,l4 = orbits(system, 2457966.5,M1=0.97*MSun,M2=3.94*MJupiter)


In [93]:
P = 2*pi*sqrt((0.449*auToMeters)**3/(G*(0.97*MSun+3.94*MJupiter)))/86400
Rsun = 695700000
Ttot = P/pi*asin(Rsun/(0.449*auToMeters)*sqrt((1+0.1)**2 - 0.75**2)/sin(89.32*pi/180))*\
sqrt(1-0.93369**2)/(1+0.93369*sin(300.5*pi/180))
print(Ttot*24)


12.993240270288322


In [56]:
rv1 = [0]*152
rv2 = [0]*152
r = [0]*152
f = [0]*152
t = range(152)
for i in t:
    l1,l2,l3,l4 = orbits(system,(2457966.5+i),M1=0.97*MSun,M2=3.94*MJupiter)
    r[i] = l1[0]
    rv1[i], rv2[i] = l4
    f[i] = l1[1]*180/pi
fig = plt.figure()
sub1 = fig.add_subplot(211)
sub2 = fig.add_subplot(212, sharex=sub1)
sub1.plot(t,rv1)
sub2.plot(t,rv2)
fig.subplots_adjust(bottom=0.2)
newax = sub1.twiny()
sub1.set_xlim(0,152)
l = ["August","September", "October", "November","December"]
llocations = [0, 31, 61, 92, 122]
newax.set_frame_on(True)
newax.patch.set_visible(False)
newax.set_xlim(sub1.get_xlim())
newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')
newax.spines['bottom'].set_position(('outward', 40))
newax.set_xticks(llocations)
newax.set_xticklabels(l)
sub1.set_ylabel("HD 80606 RV $\\frac{km}{s}$")
sub2.set_ylabel("HD 80606 b RV $\\frac{km}{s}$ ")
sub2.set_xlabel("JD - 2457966.5 (D)")
sub1.set_title("HD 80606 and HD 80606 b RV vs Time")

plt.tight_layout()
plt.show()

In [15]:
plt.plot(t,f)
#plt.xlabel()
plt.show("JD - 2457966.5 (D)")

In [57]:
def quickAndDirtyMinSearch(y,dt,tol=5e-3):#these must be greater then length 5, ideally 7
    if(dt<=0):
        raise(Exception("time interval must be nonnegative"))
    i = 2
    end = len(y)-2
    minList = []
    dt2 = dt+dt
    dts = dt2*dt2
    yp = (y[3]-y[1])/dt2
    ypp = (y[4] - 2*y[2] + y[0])/(dts)
    while i < end:
        sp = (y[i+1] - y[i-1])/dt2
        spp = (y[i+2] - 2*y[i] + y[i-2])/(dts)
        if(abs(sp-yp) < tol):
            if(spp * ypp < 0 and ypp < 0):
                minList.append(i)
        yp, ypp = sp, spp
        i+=1
    return minList

mins = quickAndDirtyMinSearch(r,1)

In [58]:

fig = plt.figure()

sub1 = fig.add_subplot(111)
sub1.plot(t,r)

for i in mins:
    sub1.plot(t[i],r[i],'h',label="t = {0}".format(t[i]))
fig.subplots_adjust(bottom=0.2)
newax = sub1.twiny()
sub1.set_xlim(0,152)
l = ["August","September", "October", "November","December"]
llocations = [0, 31, 61, 92, 122]
newax.set_frame_on(True)
newax.patch.set_visible(False)
newax.set_xlim(sub1.get_xlim())
newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')
newax.spines['bottom'].set_position(('outward', 40))
newax.set_xticks(llocations)
newax.set_xticklabels(l)
sub1.set_xlabel("JD - 2457966.5 (JD)")
sub1.set_ylabel("r$_{sky}$ (AU)")
sub1.set_title("r$_{sky}$ vs JD")
sub1.legend()
plt.show()

In [60]:
numMins = len(mins)
days = 15
segments = 5e3
betterMeasures = [[0 for i in range(days*int(segments))] for i in range(numMins)]
count = 0
tRanges = [[mins[i]-days/2+j/segments for j in range(days*int(segments))] for i in range(numMins)]
while count < numMins:
    for i in range(int(segments)*days):
        l1,l2,l3,l4 = orbits(system,\
                     (2457966.5+mins[count]-days/2+i/segments),M1=0.97*MSun,M2=3.94*MJupiter)
        betterMeasures[count][i] = l1[0]
    count+=1

In [61]:
minsImproved1 = quickAndDirtyMinSearch(betterMeasures[0],segments, tol=1e-13)
minsImproved2 = quickAndDirtyMinSearch(betterMeasures[1],segments, tol=1e-13)
minsImproved3 = quickAndDirtyMinSearch(betterMeasures[2],segments, tol=1e-13)

In [126]:
fig = plt.figure()
fig.subplots_adjust(bottom=0.2)
sub1 = fig.add_subplot(111)
sub1.plot(t,r)
sub1.set_xlim(0,152)
sub1.plot(tRanges[0],betterMeasures[0])
sub1.plot(tRanges[1],betterMeasures[1])

l = ["August","September", "October", "November","December"]
llocations = [0, 31, 61, 92, 122]
newax = sub1.twiny()
# for i in minsImproved1:
#     sub1.plot(tRanges[0][i], betterMeasures[0][i],'h')
# for i in minsImproved2:
#     sub1.plot(tRanges[1][i], betterMeasures[1][i],'h')
# for i in minsImproved2:
#     sub1.plot(tRanges[2][i], betterMeasures[2][i],'h')
newax.set_frame_on(True)
newax.patch.set_visible(False)
newax.set_xlim(sub1.get_xlim())
newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')
newax.spines['bottom'].set_position(('outward', 40))
newax.set_xticks(llocations)
newax.set_xticklabels(l)
sub1.set_xlabel("JD - 2457966.5 (JD)")
sub1.set_ylabel("r$_{sky}$ (AU)")
sub1.set_title("r$_{sky}$ vs JD")
sub1.plot([0,152],[0.00455,0.00455],'m-',label="Radius of HD 80606")
sub1.legend()
plt.tight_layout()
plt.show()

In [1]:
#calculating G mag prior to finding it in the gaia tycho data
a = -0.0257
b = -0.0924
c = -0.1623
d = 0.0090
sigma = 0.05
C2 = 9.788 - 8.6
G = a + b*C2 + c*C2**2 + d*C2**3 + 8.6
print(G)

8.250557758848


In [6]:
ra = 140.6569379644
de = 50.6037750175
obe = 23.43699
rar = ra*pi/180
der = de*pi/180
ober = obe*pi/180
lamb = atan2(sin(rar)*cos(ober) + tan(der)*sin(ober),cos(rar))
beta = asin(sin(der)*cos(ober) - cos(der)*sin(ober)*sin(rar))


In [34]:

#https://en.wikipedia.org/wiki/Position_of_the_Sun#Ecliptic_coordinates
def sunEclipticLongitude(d):
    n = d - 2451545
    L = 280.46 + 0.9856474*n
    g = 357.528 + 0.9856003*n
    while L < 0:
        L+=360
    while L > 360:
        L-=360
    while g < 0:
        g+=360
    while g > 360:
        g-=360
    return L + 1.915*sin(g*pi/180) + 0.02*sin(2*g*pi/180)

def parallaxContribution(day, ra, dec, dist):
    II = 1/dist
    ls = sunEclipticLongitude(day)
    dlambda = II*sin(ls-ra)/cos(dec)
    dbeta = II*cos(ls-ra)*sin(beta)
    return dlambda, dbeta
    
def properMotionCon(raRate, daRate, time):#expected in degree(or some equi)/yr
    time/=365.25
    return raRate*time, daRate*time

In [103]:
#assuming Gaia started observing 8/1/2017
totalDays = 365.25*5
epochs = sorted([rand.random()*totalDays for i in range(100)])


#airmax, x = dec, y = ra
#airmass = 2 limit

In [111]:
print(RA_0)
print(DEC_0)

140.65416666666667
50.603611111111114


In [127]:
#q4.1
ra1List = [0]*100
dec1List = [0]*100
counter = 0
#positions on 8/1 from the airmass calculator 
RA_0 = (9+22/60 + 37/3600)*360/24
DEC_0 = 50 + 36/60 + 13/3600
d2arc = 3600
while counter < 100:
    l1, l2, l3, l4 = orbits(system, 2457966.5+epochs[counter],M1=0.97*MSun,M2=3.94*MJupiter)
    o1, o2 = RA_0 + l2[3]*4.8481e-6, DEC_0 + l2[2]*4.8481e-6
    ra1List[counter], dec1List[counter] = o1 + rand.gauss(0.32,1)/3600e3, o2 + rand.gauss(0.32,1)/3600e3
    counter+=1

for i in range(100):
    ra1List[i] = (ra1List[i]-RA_0)*3600e3
    dec1List[i] = (dec1List[i] - DEC_0)*3600e3
fig = plt.figure()
sub1 = fig.add_subplot(311)
sub1.plot(epochs, ra1List,'o')
sub1.set_ylabel("RA - 140.654$^o$ (mas)")
sub1.set_xlabel("JD - 2457966.5 (JD)")
sub2 = fig.add_subplot(312)
sub2.set_ylabel("DEC - 50.603$^o$ (mas)")
sub2.set_xlabel("JD - 2457966.5 (JD)")
sub2.plot(epochs, dec1List,'o')
sub3 = fig.add_subplot(313)
sub3.plot(ra1List, dec1List, 'o')
sub3.set_xlabel("RA - 140.654$^o$(mas)")
sub3.set_ylabel("DEC - 50.603$^o$(mas)")
plt.tight_layout()
plt.show()

In [128]:
#lambda is ra, beta is dec
#q4.2
ra2List = [0]*100
dec2List = [0]*100
counter = 0
while counter < 100:
    l1, l2, l3, l4 = orbits(system, 2457966.5+epochs[counter],M1=0.97*MSun,M2=3.94*MJupiter)
    o1, o2 = RA_0 + l2[3]*4.8481e-6/3600e3, DEC_0 + l2[2]*4.8481e-6/3600e3
    a1, a2  = parallaxContribution(2457966.5+epochs[counter], o1, o2, 58.38)
    ra2List[counter], dec2List[counter] = o1+a1/3600+rand.gauss(0.32,1)/3600e3, o2+a2/3600+rand.gauss(0.32,1)/3600e3
    counter+=1
for i in range(100):
    ra2List[i] = (ra2List[i]-RA_0)*3600e3
    dec2List[i] = (dec2List[i] - DEC_0)*3600e3
fig = plt.figure()
sub1 = fig.add_subplot(311)
sub1.plot(epochs, ra2List,'o')
sub1.set_ylabel("RA - 140.654$^o$ (mas)")
sub1.set_xlabel("JD - 2457966.5 (JD)")
sub2 = fig.add_subplot(312)

sub2.set_ylabel("DEC - 50.603$^o$ (mas)")
sub2.set_xlabel("JD - 2457966.5 (JD)")
sub2.plot(epochs, dec2List,'o')
sub3 = fig.add_subplot(313)
sub3.plot(ra2List, dec2List,'o')
sub3.set_xlabel("RA - 140.654$^o$ (mas)")
sub3.set_ylabel("DEC - 50.603$^o$ (mas)")
plt.tight_layout()
plt.show() 

In [129]:
#lambda is ra, beta is dec
#q4.3
ra3List = [0]*100
dec3List = [0]*100
counter = 0

while counter < 100:
    l1, l2, l3, l4 = orbits(system, 2457966.5+epochs[counter],M1=0.97*MSun,M2=3.94*MJupiter)
    o1, o2 = RA_0 + l2[3]*4.8481e-6/3600e3, DEC_0 + l2[2]*4.8481e-6/3600e3
    a1, a2  = parallaxContribution(2457966.5+epochs[counter], o1, o2, 58.38)
    b1, b2 = properMotionCon(56.01, 10.643, 2457966.5+epochs[counter]-2451544.5)
    ra3List[counter]  = o1+a1/3600+rand.gauss(0.32,1)/3600e3 + b1/3600e3
    dec3List[counter] = o2+a2/3600+rand.gauss(0.32,1)/3600e3 + b2/3600e3
    counter+=1
    
for i in range(100):
    ra3List[i] = (ra3List[i]-RA_0)*3600e3
    dec3List[i] = (dec3List[i] - DEC_0)*3600e3
fig = plt.figure()
sub1 = fig.add_subplot(311)
sub1.plot(epochs, ra3List,'o')
sub1.set_ylabel("RA - 140.654$^o$ (mas)")
sub2 = fig.add_subplot(312)
sub2.plot(epochs, dec3List,'o')
sub2.set_ylabel("DEC - 50.603$^o$ (mas)")
sub1.set_xlabel("JD - 2457966.5 (JD)")
sub2.set_xlabel("JD - 2457966.5 (JD)")
sub3 = fig.add_subplot(313)
sub3.plot(ra3List, dec3List,'o')
sub3.set_xlabel("RA - 140.654$^o$ (mas)")
sub3.set_ylabel("DEC - 50.603$^o$ (mas)")
plt.tight_layout()
plt.show() 

#HIP id: 45982
#epoch 2015.0, I interpreted this as 2015? but this might be wrong
ra = 140.6569379644
err = 0.155 #mas
de = 50.6037750175
eerd = 0.297 #mas
para = 15.33
errp - 0.26
pmRA = 56.01 #mas/yr
errpm = 0.404
pmDe = 10.643
distance = 190.41 yl
right acsension = phi
declination = theta(but measured from the plane not the pole)


lambda is the star we're observing's ecliptic longitude
lambda_s is the ecliptic longitude of the sun
Beta is a shift in the ecliptic lattitude
PI = a_earth/r(distance to star) in radians
fromula:
delta lambda* cos(Beta) = PI*sin(lambda_s - lambda)
delta Beta = PI*cos(lambda_s -lambda)sin(Beta)

ecliptic obquity = 23.4 deg

In [ ]:
def eulersMethod(fprime, t0, y0, h, tEnd):
    tRange = [t0 + (tEnd - t0)*h*i for i in range(int(1/h))]
    returnList = [0]*len(tRange)
    returnList[0] = y0
    for i in range(1,len(tRange)):
        yi, ti = returnList[i-1], tRange[i-1]
        returnList[i] = yi + h/2*(fprime(ti,yi) + \
                    fprime(tRange[i], yi+h*fprime(ti,yi)))
    return tRange, returnList